In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

In [2]:
fnames = !ls ../all_bills/original/

In [43]:
origs = [
    cv2.resize(
        cv2.imread('../all_bills/original/' + f), (240,320)
    )[:,:,::-1]
        for f in tqdm(fnames)]
masks = [
    cv2.resize(
        cv2.imread('../all_bills/processed/' + f), (240,320)
    )[:,:,0] for f in tqdm(fnames)
        ]

HBox(children=(IntProgress(value=0, max=304), HTML(value='')))

HBox(children=(IntProgress(value=0, max=304), HTML(value='')))

In [44]:
from image_segmentation_functions import Segmenter

In [45]:
s = Segmenter()

In [88]:
a = s._get_mask(origs[0])

In [84]:
def grabcut_segment(img):
    mask = np.zeros(img.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (10,0,220,320)
    cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    return mask2

In [93]:
segm_nn = [(s._get_mask(x) > 0.7).astype('uint8') for x in tqdm(origs)]
segm_gc = [grabcut_segment(x) for x in tqdm(origs)]

HBox(children=(IntProgress(value=0, max=304), HTML(value='')))

HBox(children=(IntProgress(value=0, max=304), HTML(value='')))

In [113]:
def IOU(mask, pred):
    inter = np.sum((mask > 100) & pred)
    union = np.sum((mask > 100) | pred)
    return inter / union

In [121]:
iou_nn = [IOU(*mp) for mp in zip(masks,segm_nn)]
iou_gc = [IOU(*mp) for mp in zip(masks,segm_gc)]

In [126]:
print(f"U-net IoU: {np.mean(iou_nn)}")
print("U-net speed: 8.47 it/s")

U-net IoU: 0.9835009858258805
U-net speed: 8.47 it/s


In [127]:
print(f"GrabCut IoU: {np.mean(iou_gc)}")
print("GrabCut speed: 1.32 it/s")

GrabCut IoU: 0.8797207280045801
GrabCut speed: 1.32 it/s
